In [1]:
## Script to compare distributions in simulated data to distributions in real data

# Libraries

In [6]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'"


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"


In [7]:
source('MS4_Plot_Config.r')

# Parameters

In [ ]:
### Parameters for co-EQTL data

In [58]:
### Path to saved simulation results from previous script
output_path_sc = "../results/current/A1/"

In [59]:
### Path to summary statistics of real single-cell input datasets
data_path_summary_stats = '../data/current/summary_stats/'

## Files: Sample_gene_statistic  (info about genes on sample level)
#         Sample_summary (information about samples)
#         Sample_cell statistic

In [281]:
#### Simulated variant to load

In [282]:
coeqtl_variant = 'scaling'

In [310]:
variant = 'wijstCD4_T3'

In [311]:
correlations_var = ''

In [ ]:
### Specify which real dataset to load (should match simulated dataset)

In [312]:
dataset = 'wijst2018'
cell_type = 'CD4_T'

# Functions

In [313]:
source('MS1_Functions.r')

# Data

## Load summaries from real single-cell input datasets

In [314]:
datasets = dataset

In [315]:
cell_types =  cell_type

In [316]:
## get for each sample and gene of dataset summary information, like mean expression, percentage zero, ...

### Sample summaries

In [317]:
sample_summaries = data.frame()

In [318]:
for(i in datasets){
    for (j in cell_types){
    data = read.csv(paste0(data_path_summary_stats, 'D1_Sample_summarywg3_', i ,  j, '.Qced.Normalized.SCs.Rds.csv'))
    data$dataset = i
    sample_summaries = rbind(data, sample_summaries)
    }
    }

#D1_Sample_summarywg3_wijst2018CD4_T.Qced.Normalized.SCs.Rds.csv

In [319]:
head(sample_summaries,2)

,X,Assignment,amount_cells,dataset,cell_type
,<int>,<chr>,<int>,<chr>,<chr>
1,1,LL001060,221,wijst2018,CD4_T.Qced.Normalized.SCs.Rds
2,2,LL002176,338,wijst2018,CD4_T.Qced.Normalized.SCs.Rds


In [320]:
#sample_summaries$cell_type_short = str_replace(sample_summaries$cell_type, '\\..*', '')

sample_summaries$cell_type_short = str_replace(sample_summaries$cell_type, 'Qced.Normalized.SCs.Rds', '')

In [321]:
unique(sample_summaries$cell_type_short)

[1] "CD4_T."

In [322]:
## Check for uniqueness
head(sample_summaries,2)

,X,Assignment,amount_cells,dataset,cell_type,cell_type_short
,<int>,<chr>,<int>,<chr>,<chr>,<chr>
1,1,LL001060,221,wijst2018,CD4_T.Qced.Normalized.SCs.Rds,CD4_T.
2,2,LL002176,338,wijst2018,CD4_T.Qced.Normalized.SCs.Rds,CD4_T.


In [323]:
sample_summaries$Assignment = str_replace(sample_summaries$Assignment, ';;.*', '')

In [324]:
## Summarise the amount of cells per dataset and cell-type as table

In [325]:
unique(sample_summaries[,c('Assignment', 'amount_cells', 'dataset', 'cell_type', 'cell_type_short')]) %>% group_by(cell_type_short, dataset) %>% summarise(amount_individuals = length(unique(Assignment)),
                                                                      mean_amount_cells = mean(amount_cells),
                                                                      sd_amount_cells = sd(amount_cells)) %>% arrange(mean_amount_cells)

`summarise()` has grouped output by 'cell_type_short'. You can override using
the `.groups` argument.


cell_type_short,dataset,amount_individuals,mean_amount_cells,sd_amount_cells
<chr>,<chr>,<int>,<dbl>,<dbl>
CD4_T.,wijst2018,41,262.2195,125.6834


### Sample Gene statistics

In [326]:
sample_gene_statistic = data.frame()
sample_gene_statistic_trans = data.frame()  # tBD

In [327]:
for(i in datasets){
    for (j in cell_types){
        summary_stats_name = paste0("D1_Sample_gene_statisticwg3_",i ,  j, ".Qced.Normalized.SCs.Rds.csv")   # without data for raw count based / for current threshold doesn't matter
        if(file.exists(paste0(data_path_summary_stats, summary_stats_name))){
            gene_sample_stats = read.csv(paste0(data_path_summary_stats, summary_stats_name))
            gene_sample_stats = setDT(gene_sample_stats)
            sample_gene_statistic = rbind(gene_sample_stats, sample_gene_statistic)             
                }
            }
        }
    
        

In [328]:
head(sample_gene_statistic,2)

X,gene,amount_cells,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,amount_non_zero_counts,amount_cells_bin,percentage_zero_bin,sample,cell_type,dataset
<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,NOC2L,221,189,0.14932127,2,0,0.13669272,85.52036,32,"(100,250]","(80,90]",LL001060,CD4_T.Qced.Normalized.SCs_Dan.Rds,wg3_wijst2018
2,HES4,221,209,0.06334842,2,0,0.07778692,94.57014,12,"(100,250]","(90,95]",LL001060,CD4_T.Qced.Normalized.SCs_Dan.Rds,wg3_wijst2018


### Sample_cell_statistics

In [329]:
sample_cell_statistic = data.frame()

In [330]:
for(i in datasets){
    for (j in cell_types){
        summary_stats_name = paste0("D1_Sample_cell_statisticwg3_",i ,  j, ".Qced.Normalized.SCs.Rds.csv")   # without data for raw count based / for current threshold doesn't matter
        if(file.exists(paste0(data_path_summary_stats, summary_stats_name))){
            gene_sample_stats = read.csv(paste0(data_path_summary_stats, summary_stats_name))
            gene_sample_stats = setDT(gene_sample_stats)
            sample_cell_statistic = rbind(gene_sample_stats,  sample_cell_statistic)             
                }
            }
        }
    

In [331]:
head(sample_cell_statistic,2)

X,cell,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,amount_non_zero_counts,sample,cell_type,dataset
<int>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>
1,AAACCTGGTGCCTTGG_pilot3_lane1,2078,19422,18516,0.10699207,180,0,2.4715344,0.9533519,906,LL001060,CD4_T.Qced.Normalized.SCs.Rds,wg3_wijst2018
2,AAAGCAACACATCCGG_pilot3_lane1,1303,19422,18973,0.06708887,85,0,0.9660462,0.9768819,449,LL001060,CD4_T.Qced.Normalized.SCs.Rds,wg3_wijst2018


## Load simulated data

In [332]:
### counts
count_data = fread(paste0(output_path_sc, 'A1_4_Simulated_Counts' , variant, coeqtl_variant , '.csv.gz'))

In [333]:
head(count_data,2)

V1,variable,cell_ident,value,scaling,mu,size,counts,basemean,ncells,ind
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<chr>
1,gene1_1,cell_XX;1_149_1,0.9802071,0.9754055,1,12,4,1,149,XX;1
2,gene1_1,cell_XX;1_149_39,0.3085414,0.7945828,1,12,1,1,149,XX;1


In [334]:
### Summary stats of simulated data

In [335]:
gene_summary = fread(paste0(output_path_sc, 'A1_4_Gene_Summary_' ,variant, '_', coeqtl_variant , '.csv.gz'))

In [336]:
head(gene_summary,2)

V1,variable,basemean,ncells,ind,amount_zero_counts,amount_cells,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,percentage_zero_bin
<int>,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
1,gene1_1,1,149,XX;1,53,149,1.006711,5,0,1.033738,35.57047,"(20,40]"
2,gene2_1,16,149,XX;1,0,149,16.127517,42,3,55.760657,0.00000,"(-0.001,5]"


In [337]:
## binarize amount of cells
gene_summary = gene_summary %>% mutate(amount_cells_bin = cut(amount_cells, breaks=c(0,25, 50, 100,250, 500, 750, 1500)))

In [338]:
cell_summary = fread(paste0(output_path_sc, 'A1_4_Cell_Summary_' ,variant, '_', coeqtl_variant , '.csv.gz'))

In [339]:
head(cell_summary,2)

V1,ncells,ind,cell_ident,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,var_counts,percentage_zero,total_mean_counts,scaling
<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,149,XX;1,cell_XX;1_149_1,101937,20000,7948,5.09685,61,0,53.19423,0.3974,5.865534,1.150815
2,149,XX;1,cell_XX;1_149_10,111749,20000,7262,5.58745,55,0,61.15941,0.3631,5.865534,1.049769


# Compare distributions/ investigate relationships

#### Gene level relationships

In [340]:
### Combine with real data for comparison

In [341]:
head(sample_gene_statistic,2)

X,gene,amount_cells,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,amount_non_zero_counts,amount_cells_bin,percentage_zero_bin,sample,cell_type,dataset
<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,NOC2L,221,189,0.14932127,2,0,0.13669272,85.52036,32,"(100,250]","(80,90]",LL001060,CD4_T.Qced.Normalized.SCs_Dan.Rds,wg3_wijst2018
2,HES4,221,209,0.06334842,2,0,0.07778692,94.57014,12,"(100,250]","(90,95]",LL001060,CD4_T.Qced.Normalized.SCs_Dan.Rds,wg3_wijst2018


In [342]:
# Simulated data summary

In [343]:
head(gene_summary,2)

V1,variable,basemean,ncells,ind,amount_zero_counts,amount_cells,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,percentage_zero_bin,amount_cells_bin
<int>,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>,<fct>
1,gene1_1,1,149,XX;1,53,149,1.006711,5,0,1.033738,35.57047,"(20,40]","(100,250]"
2,gene2_1,16,149,XX;1,0,149,16.127517,42,3,55.760657,0.00000,"(-0.001,5]","(100,250]"


In [344]:
gene_summary$type = 'simulated'
sample_gene_statistic$type = 'real'

In [345]:
random_sample_subset = unique(sample_gene_statistic$sample) # decide whether one or many samples

In [346]:
head(sample_gene_statistic,2)

X,gene,amount_cells,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,amount_non_zero_counts,amount_cells_bin,percentage_zero_bin,sample,cell_type,dataset,type
<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,NOC2L,221,189,0.14932127,2,0,0.13669272,85.52036,32,"(100,250]","(80,90]",LL001060,CD4_T.Qced.Normalized.SCs_Dan.Rds,wg3_wijst2018,real
2,HES4,221,209,0.06334842,2,0,0.07778692,94.57014,12,"(100,250]","(90,95]",LL001060,CD4_T.Qced.Normalized.SCs_Dan.Rds,wg3_wijst2018,real


In [347]:
### combine simulated + real dataset
plot_data = rbind(gene_summary[,c('mean_counts', 'variance_counts','amount_cells_bin' , 'type', 'percentage_zero', 'max_counts')],
                  sample_gene_statistic[sample_gene_statistic$sample %in% random_sample_subset[1:3],c('mean_counts', 'variance_counts','amount_cells_bin',  'type' , 'percentage_zero', 'max_counts')])

In [348]:
### Investigate mean variance relationship

In [349]:
### Investigate mean variance relationship combined
options(repr.plot.width=10, repr.plot.height=10)
pdf(paste0('figures/A1_10/Mean_Variance_Combined_Fixed_Parameters', variant, coeqtl_variant, '.pdf'), width =2, height =2)
p1 = ggplot(plot_data, aes(x=log(mean_counts+1), y=log(variance_counts+1), color = type)) + 
    geom_point(size = 0.1, alpha = 0.5)  +ylim(0,15) + xlim(0,10) + plot_config + theme(legend.position = 'None')#  facet_wrap( amount_cells_bin~.  )

print(p1)

p2 = ggplot(plot_data, aes(x=log(mean_counts+1), y=log(variance_counts+1), color = type)) + 
    geom_point(size = 0.1, alpha = 0.5)  +ylim(0,15) + xlim(0,10) + facet_wrap( .~ type  ) + plot_config 

print(p2)
dev.off()

ggsave(plot = p1, filename= paste0('figures/A1_10/Mean_Variance_Combined_Fixed_Parameters', variant, coeqtl_variant, '.tiff'), dpi=300)

pdf 
  2

Saving 7 x 7 in image


In [350]:
#### Mean zero relationship on simulated data

In [351]:
#### Mean zero relationship combined

pdf(paste0('figures/A1_10/Mean_Zero_Combined_Fixed_Parameters',variant, coeqtl_variant, '.pdf'), width =2, height =2)
p= ggplot(plot_data, aes(x=percentage_zero, y=log(mean_counts+1), color = type)) + 
    geom_point(size = 0.1, alpha = 0.5)  + ylim(0,10) + plot_config + theme(legend.position = 'None')# +  facet_wrap( amount_cells_bin~ . ) 

print(p)

p= ggplot(plot_data, aes(x=percentage_zero, y=log(mean_counts+1), color = type)) + 
    geom_point(size = 0.1, alpha = 0.5) + ylim(0,10) +facet_wrap( .~ type  ) + plot_config

print(p)

dev.off()

# take twice log to see curve as in splatter package

pdf 
  2

In [352]:
#### Max zero relationship

In [353]:
#### Max zero relationship combined

pdf(paste0('figures/A1_10/Max_Counts_Zero_Combined_Fixed_Parameters', variant, coeqtl_variant, '.pdf'), width =2, height =2)
p = ggplot(plot_data, aes(x=percentage_zero, y=log(max_counts+1), color = type)) + 
    geom_point(size = 0.1, alpha = 0.5) + ylim(0,10) + plot_config+ theme(legend.position = 'None') #  +  facet_wrap( amount_cells_bin~ . ) 
print(p)

p = ggplot(plot_data, aes(x=percentage_zero, y=log(max_counts+1), color = type)) + 
    geom_point(size = 0.1, alpha = 0.5)  + ylim(0,10)  +    facet_wrap( .~ type ) + plot_config
print(p)

dev.off()

pdf 
  2

In [354]:
###### Variance zero relationship simulation

In [355]:
#### Variance zero relationship combined
options(repr.plot.width=30, repr.plot.height=30)
pdf(paste0('figures/A1_10/Variance_Zero_Combined_Fixed_Parameters',  variant, coeqtl_variant,  '.pdf'), width =2, height =2)
p= ggplot(plot_data, aes(x=percentage_zero, y=log(variance_counts+1), color = type)) + 
    geom_point(size = 0.1, alpha = 0.5)  + ylim(0,15) + plot_config + theme(legend.position = 'None')# +   facet_wrap(amount_cells_bin~ .)
print(p)


p= ggplot(plot_data, aes(x=percentage_zero, y=log(variance_counts+1), color = type)) + 
    geom_point(size = 0.1, alpha = 0.5)  + ylim(0,15)  +   facet_wrap(.~ type ) + plot_config
print(p)

dev.off()

pdf 
  2

#### Cell level relationships

In [233]:
head(cell_summary,2) # simulated data

V1,ncells,ind,cell_ident,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,var_counts,percentage_zero,total_mean_counts,scaling
<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,14,XX;1,cell_XX;1_14_1,88060,20000,8177,4.40300,47,0,40.38871,0.40885,5.377257,1.2212712
2,14,XX;1,cell_XX;1_14_10,115329,20000,7162,5.76645,50,0,63.06186,0.35810,5.377257,0.9325074


In [234]:
### test filtering out cells based on scaling factor in simulated data
nrow(cell_summary)
nrow(cell_summary[(cell_summary$scaling > 0.5) & (cell_summary$scaling < 2),])

nrow(cell_summary[(cell_summary$scaling > 0.5) & (cell_summary$scaling < 2),])/ nrow(cell_summary)

[1] 1279

[1] 1273

[1] 0.9953088

In [62]:
### test filtering out cells based on scaling fator in real data
nrow(sample_cell_statistic)
nrow(sample_cell_statistic[(sample_cell_statistic$scaling_factor < 2 ) & (sample_cell_statistic$scaling_factor > 0.5 ),])
nrow(sample_cell_statistic[(sample_cell_statistic$scaling_factor < 2 ) & (sample_cell_statistic$scaling_factor > 0.5 ),]) /nrow(sample_cell_statistic)
# filter out around 7% of cells

[1] 663

[1] 0

[1] 0

In [63]:
head(sample_cell_statistic,2)

X,cell,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,amount_non_zero_counts,sample,cell_type,dataset
<int>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>
1,AAAGCAAGTCGCATCG_pilot3_lane1,5216,14699,13237,0.3548541,322,0,16.401625,0.9005375,1462,LL007985,B.Qced.Normalized.SCs.Rds,wg3_wijst2018
2,AACTTTCTCACAATGC_pilot3_lane1,3848,14699,13536,0.2617865,217,0,8.229191,0.9208790,1163,LL007985,B.Qced.Normalized.SCs.Rds,wg3_wijst2018


In [64]:
head(cell_summary,2)

V1,ncells,ind,cell_ident,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,var_counts,percentage_zero,total_mean_counts,scaling
<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,10,XX;1,cell_XX;1_10_1,117295,20000,8459,5.86475,613,0,332.1105,0.42295,6.408285,1.092678
2,10,XX;1,cell_XX;1_10_10,94422,20000,7796,4.72110,403,0,186.6399,0.38980,6.408285,1.357371


In [65]:
max(cell_summary$amount_counts)

[1] 567109

In [66]:
### Distribution of counts per sample and cell-type
options(repr.plot.width=30, repr.plot.height=30)
pdf(paste0('figures/A1_10/Distribution_Counts_Simulation',  variant, coeqtl_variant, '.pdf'), width =10, height =10)
p = ggplot(cell_summary, aes(x=amount_counts)) + 
     geom_histogram() + facet_wrap(ncells + ind ~ ., scales = 'free') + theme(text = element_text(size =20)) + xlim(0,max(cell_summary$amount_counts)) + plot_config
print(p)
dev.off()

`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
Warning message:
"Removed 90 rows containing missing values or values outside the scale range
(`geom_bar()`)."


pdf 
  2

In [67]:
### Correlation calc + defined scaling factor

In [68]:
total_mean = cell_summary %>% group_by(ncells, ind) %>% summarise(total_mean_counts = mean(mean_counts))
cell_summary = merge(cell_summary, total_mean)
cell_summary$scaling_calc = cell_summary$total_mean_counts / cell_summary$mean_counts 

`summarise()` has grouped output by 'ncells'. You can override using the
`.groups` argument.


In [69]:
head(cell_summary,2)

ncells,ind,total_mean_counts,V1,cell_ident,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,var_counts,percentage_zero,scaling,scaling_calc
<int>,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
10,XX;1,6.408285,1,cell_XX;1_10_1,117295,20000,8459,5.86475,613,0,332.1105,0.42295,1.092678,1.092678
10,XX;1,6.408285,2,cell_XX;1_10_10,94422,20000,7796,4.72110,403,0,186.6399,0.38980,1.357371,1.357371


In [70]:
cor(cell_summary$scaling, cell_summary$scaling_calc)

1


In [71]:
## Relationship Count and Amount Zero Relationship

In [72]:
pdf(paste0('figures/A1_10/Cell_Zero_Count_Relationship_Simulation', variant, coeqtl_variant, '.pdf'), width =10, height =5)
p =ggplot(cell_summary , aes(x=amount_counts, y=amount_zero_counts, color= ncells)) + 
    geom_point(size = 0.1) + theme(text = element_text(size = 20)) + facet_wrap(ncells ~ ., scales = 'free')  + plot_config
print(p)
dev.off()
# consider: different amount of genes per cell-type in data!

pdf 
  2

In [73]:
pdf(paste0('figures/A1_10/Cell_Zero_Percentage_Count_Relationship_Simulation',  variant, coeqtl_variant, '.pdf'), width =10, height =5)
p =ggplot(cell_summary , aes(x=amount_counts, y=percentage_zero, color= ncells)) + 
    geom_point(size = 0.1) + theme(text = element_text(size = 20)) + facet_wrap(ncells ~ ., scales = 'free')  + ylim(0,1) + plot_config
print(p)
dev.off()
# consider: different amount of genes per cell-type in data!
# here: scaling factor needs to be calculated per cell-type as differing amount of genes

pdf 
  2

In [74]:
### Relationship Scaling Factor / Counts

In [75]:
head(cell_summary,2)

ncells,ind,total_mean_counts,V1,cell_ident,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,var_counts,percentage_zero,scaling,scaling_calc
<int>,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
10,XX;1,6.408285,1,cell_XX;1_10_1,117295,20000,8459,5.86475,613,0,332.1105,0.42295,1.092678,1.092678
10,XX;1,6.408285,2,cell_XX;1_10_10,94422,20000,7796,4.72110,403,0,186.6399,0.38980,1.357371,1.357371


In [76]:
pdf(paste0('figures/A1_10/Cell_Scaling_Factor_Count_Relationship_Simulation',  variant, coeqtl_variant, '.pdf'), width =10, height =5)
p =ggplot(cell_summary , aes(x=amount_counts, y=scaling, color=ncells)) + 
    geom_point(size = 0.1) + theme(text = element_text(size = 20)) + facet_wrap(ncells ~ ., scales = 'free') + plot_config
print(p)
p =ggplot(cell_summary , aes(x=amount_counts, y=scaling_calc, color=ncells)) + 
    geom_point(size = 0.1) + theme(text = element_text(size = 20)) + facet_wrap(ncells ~ ., scales = 'free') + ggtitle('calculated_scaling') + plot_config
print(p)
dev.off()

pdf 
  2

In [77]:
### Relationship Scaling Factor / Percentage Zero

In [78]:
pdf(paste0('figures/A1_10/Cell_Scaling_Factor_Percentage_Zero_Relationship_Simulation',  variant, coeqtl_variant, '.pdf'), width =10, height =5)
p =ggplot(cell_summary , aes(x=percentage_zero, y=scaling, color=ncells)) + 
    geom_point(size = 0.1) + theme(text = element_text(size = 20)) + facet_wrap(ncells~ ., scales = 'free')  + plot_config
print(p)
p =ggplot(cell_summary , aes(x=percentage_zero, y=scaling_calc, color=ncells)) + 
    geom_point(size = 0.1) + theme(text = element_text(size = 20)) + facet_wrap(ncells~ ., scales = 'free') + ggtitle('calculated scaling') + plot_config
print(p)
dev.off()

pdf 
  2

In [79]:
### Distribution mean counts

In [80]:
pdf(paste0('figures/A1_10/Distribution_Mean_Counts_Simulation',  variant, coeqtl_variant, '.pdf'), width =10, height =5)
p = ggplot(cell_summary , aes(x=mean_counts)) + 
     geom_histogram() + facet_wrap(ncells + ind ~ ., scales = 'free') + theme(text = element_text(size =20))  + plot_config
print(p)
dev.off()

`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


pdf 
  2